# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM

## Important Notes

- Make sure you push on GitHub your Notebook with all the cells already evaluated
- Note that maps do not render in a standard Github environment : you should export them to HTML and link them in your notebook.
- Remember that `.csv` is not the only data format. Though they might require additional processing, some formats provide better encoding support.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
- Please write all your comments in English, and use meaningful variable names in your code

## Background

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.

## Assignment

1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

In [1]:
import os
import pandas as pd
import json
import folium
import branca
folium.__version__ == '0.5.0'

True

In [2]:
swiss_2017_f = pd.read_csv('Data/2_1 Taux de chômage.csv',header=1,quotechar='"',encoding='utf-16le')
swiss_2017_d = pd.read_csv('Data/2_1 Arbeitslosenquoten.csv',header=1,quotechar='"',encoding='utf-16le')
swiss_2017_i = pd.read_csv('Data/2_1 Tasso di disoccupazione.csv',header=1,quotechar='"',encoding='utf-16le')
swiss_2017_f.drop('Mois',1, inplace=True)
swiss_2017_d.drop('Monat',1, inplace=True)
swiss_2017_i.drop('mese',1, inplace=True)
swiss_2017_f.drop(0,0, inplace=True)
swiss_2017_d.drop(0,0, inplace=True)
swiss_2017_i.drop(0,0, inplace=True)


In [3]:
swiss_2017 = swiss_2017_f
swiss_2017.rename(columns={'Canton':'Canton (f)'},inplace=True)
swiss_2017['Canton (d)']=swiss_2017_d['Kanton']
swiss_2017['Canton (i)']=swiss_2017_i['Cantone']
swiss_2017 = swiss_2017[['Canton (f)','Canton (d)','Canton (i)','Total']]
swiss_2017['Total']=swiss_2017['Total'].astype(float)
swiss_2017.reset_index(drop=True,inplace=True)
swiss_2017

/home/samuel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Canton (f),Canton (d),Canton (i),Total
0,Zurich,Zürich,Zurigo,3.6
1,Berne,Bern,Berna,2.7
2,Lucerne,Luzern,Lucerna,1.9
3,Uri,Uri,Uri,1.0
4,Schwyz,Schwyz,Svitto,1.8
5,Obwald,Obwalden,Obwaldo,0.9
6,Nidwald,Nidwalden,Nidwaldo,1.1
7,Glaris,Glarus,Glarona,2.2
8,Zoug,Zug,Zugo,2.4
9,Fribourg,Freiburg,Friburgo,2.7


In [4]:
canton_topo_path = os.path.join('topojson','ch-cantons.topojson.json')
topo_json_data = json.load(open(canton_topo_path))
topo_json_data

{'arcs': [[[6286, 7049],
   [-33, 10],
   [-114, -55],
   [-60, -45],
   [-9, -15],
   [-13, 10],
   [-2, -5],
   [0, -5],
   [0, -5],
   [0, -6],
   [-2, 0],
   [-5, -5],
   [-4, 0],
   [-5, 0],
   [-2, 0],
   [-2, 0],
   [0, -5],
   [-20, -45],
   [-6, -10],
   [-3, -5],
   [-2, -25],
   [2, -10],
   [3, -15],
   [8, -10],
   [9, -10],
   [5, -15],
   [0, -15]],
  [[6031, 6763],
   [-9, -5],
   [-9, 0],
   [-13, -10],
   [-7, -10],
   [-20, 5],
   [-8, 5],
   [-3, 25],
   [0, 25],
   [-2, 0],
   [-2, 0],
   [-2, 0],
   [-5, -5],
   [-2, 0],
   [-2, 0],
   [-2, 5],
   [-3, 0],
   [-4, -5],
   [-9, 0],
   [-2, 0],
   [-2, -5],
   [-2, 0],
   [-5, 0],
   [-2, 5],
   [0, 5],
   [-2, 5],
   [-7, 0],
   [-2, 0],
   [-4, 0],
   [-3, 0],
   [-4, 0],
   [-2, 0],
   [-2, 0],
   [-5, 0],
   [-2, 5],
   [-2, 0],
   [0, 10],
   [2, 5],
   [0, 5],
   [2, 0],
   [2, 5],
   [3, 5],
   [0, 5],
   [-5, 5],
   [0, 5],
   [-2, 10],
   [-4, 10],
   [-5, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   [0, 5],
   

In [5]:
N_canton = len(topo_json_data['objects']['cantons']['geometries'])

swiss_2017['id']=swiss_2017['Canton (f)']

for i in range(N_canton):
    swiss_2017['id'].loc[i] = topo_json_data['objects']['cantons']['geometries'][i]['id']

#for i in range(N_canton):
#    topo_json_data['objects']['cantons']['geometries'][i]['properties']['name']=swiss_2017['Canton (f)'].loc[i]

/home/samuel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/samuel/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/samuel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

In [6]:
center_coord = [46.8011111,8.2266667]

swiss_map = folium.Map(location=center_coord,
            tiles='cartodbpositron',           
            zoom_start=8)

#folium.TopoJson(topo_json_data,'objects.cantons').add_to(swiss_map)
#swiss_map

In [7]:
total_serie = swiss_2017.set_index('id')['Total']
colorscale = branca.colormap.linear.YlOrRd.scale(0, 6)

def style_function(feature):
    unemployed = total_serie.get(feature['id'], None)
    return {
        'fillOpacity': 0.5,
        'weight': 0.5,
        'fillColor': '#black' if unemployed is None else colorscale(unemployed)
    }


folium.TopoJson(topo_json_data,
               'objects.cantons',
                style_function=style_function
                ).add_to(swiss_map)

In [8]:
swiss_map